In [1]:
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer,porter
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import pandas
import numpy as np
import re
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.ensemble import RandomForestRegressor as rfr
from scipy.stats import spearmanr as spcorr,pearsonr as pcorr
from IPython.display import display
import matplotlib.pyplot as plt
%pylab inline
pylab.rcParams['figure.figsize'] = (10, 6)

Populating the interactive namespace from numpy and matplotlib


In [185]:
def maxCase(x):
    result = [i for (i,v) in zip(range(-2,3),[y==max(x) for y in x]) if v]
    if(len(result) > 1):
        s = sum(result)
        if(s < -2):
            return -2
        elif(s > 2):
            return 2
        else:
            return s
    else:
        return result[0]
    
def worstCase(x):
    if x['vneg']>0:
        return -2
    elif x['neg']>0:
        return -1
    elif x['neutral']>0:
        return 0
    elif x['pos']>0:
        return 1
    else:
        return 2
    
def bestCase(x):
    if x['vpos']>0:
        return 2
    elif x['pos']>0:
        return 1
    elif x['neutral']>0:
        return 0
    elif x['neg']>0:
        return -1
    else:
        return -2

sentiments = pandas.read_csv('review_sentiment_coreNLP.txt')
sentiments_R = pandas.read_csv('Score_of_reviews.csv')['scores']
scores = sentiments.apply(maxCase,axis=1)

In [186]:
reviews = pandas.read_pickle('consumer_affairs_sallie_mae.data')
reviews['sentiment'] = scores
reviews.sentiment.value_counts()

-1    1273
 0      23
-2      10
 1       4
Name: sentiment, dtype: int64

In [187]:
def noPunct(txt):
    for punct in string.punctuation:
        txt = txt.replace(punct,'')
    return txt

def noNumber(txt):
    return re.sub(r'[$\s]*\d+\.?\d*','',txt)

def LemmaTokenizer(doc):
    wnl = WordNetLemmatizer()
    lemmas =[wnl.lemmatize(t) for t in word_tokenize(doc)]
    lemmas = [noPunct(l) for l in lemmas]
    lemmas = [noNumber(l) for l in lemmas]
    lemmas = list(filter(lambda x: len(x) > 0,lemmas))
    return lemmas

def StemmerTokenizer(doc):
    port = porter.PorterStemmer()
    tokens =[port.stem(t) for t in word_tokenize(doc)]
    tokens = [noPunct(l) for l in tokens]
    tokens = [noNumber(l) for l in tokens]
    tokens = list(filter(lambda x: len(x) > 0,tokens))
    return tokens

extra_stopwords = stopwords.words('english') + ['sallie','mae','loan','nt','va','le','ha']

In [230]:
tfidf = TfidfVectorizer(lowercase=True, analyzer='word', stop_words=extra_stopwords,tokenizer=LemmaTokenizer,
                        ngram_range=(1,2), binary=False, norm='l1', use_idf=True, smooth_idf=True, sublinear_tf=True,
                       max_features=None,min_df=1,max_df=0.99)
data = tfidf.fit_transform(reviews.review)

In [231]:
model = rfc(n_estimators=1000,criterion='entropy',max_features='auto',bootstrap=False,oob_score=False,n_jobs=6,
           random_state=20170202,verbose=1)
model.fit(data,reviews.sentiment)

[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.5s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:    1.4s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:    2.6s
[Parallel(n_jobs=6)]: Done 1000 out of 1000 | elapsed:    3.3s finished


RandomForestClassifier(bootstrap=False, class_weight=None,
            criterion='entropy', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=6,
            oob_score=False, random_state=20170202, verbose=1,
            warm_start=False)

In [232]:
features = tfidf.get_feature_names()
importance = model.feature_importances_
feature_summary = pandas.DataFrame({'name':features,'value':list(importance)})
feature_summary.sort_values('value',ascending=False,inplace=True)

In [233]:
n = 25
ngrams = feature_summary.head(n).index

In [234]:
rho = []
pval = []
for a in ngrams:
    r,p = spcorr(data[:,a].toarray().flatten(),reviews.sentiment.values)
    rho.append(r)
    pval.append(p)
    del r,p

In [235]:
corr_summary = pandas.DataFrame({'name':feature_summary.head(n).name,'cor':rho,'pval':pval})
pandas.options.display.max_rows=n
corr_summary.sort_values('cor')

cor               name          pval
64108  -0.234205  offer refinancing  8.817950e-18
76829  -0.122418        refinancing  8.860113e-06
102888 -0.083655              would  2.443787e-03
64061  -0.062711              offer  2.321785e-02
67466  -0.033691                pay  2.229946e-01
36113  -0.022423               fact  4.174111e-01
92041  -0.019848             talked  4.729114e-01
74945  -0.018029               rate  5.144195e-01
52452  -0.016647               late  5.471773e-01
89744  -0.004755            student  8.634887e-01
45326  -0.003057               help  9.119919e-01
76974  -0.003037             refuse  9.125413e-01
95002  -0.002254               told  9.350375e-01
11920  -0.002045              asked  9.410460e-01
103819  0.007168               year  7.954859e-01
64576   0.017422                one  5.286921e-01
71287   0.018542             please  5.025151e-01
99864   0.020577               want  4.568058e-01
93172   0.026254              thing  3.423685e-01
45737   0.036387               high  1.881190e-01
78186   0.037527             report  1.746475e-01
94138   0.038643               time  1.621682e-01
49579   0.038951           interest  1.588462e-01
34814   0.048495              every  7.933348e-02
26745   0.104545           daughter  1.502468e-04

In [229]:
cormat = np.corrcoef(np.transpose(data[:,ngrams].todense()))
cormat_df = pandas.DataFrame(cormat,columns=feature_summary.loc[ngrams,'name'],index=feature_summary.loc[ngrams,'name'])
cormat_df.to_excel('ngram_corr.xlsx')

In [518]:
mask = [x > 0 for x in data[:,84569].toarray().flatten()]

display(reviews.iloc[mask])

plt.scatter(data[:,84569].toarray().flatten(),reviews.sentiment)
plt.xlabel('offer refinancing')
plt.ylabel('sentiment')
plt
plt.show()

datetime             location       name  rating  \
1    2017-01-03 16:46:44  Stevenson Ranch, CA        Joe     1.0   
3    2016-11-08 23:55:28        Knoxville, TN      K. L.     1.0   
4    2016-10-30 21:13:14         San Jose, CA      M. A.     1.0   
6    2016-10-25 21:57:07        Milwaukee, WI       Tony     1.0   
10   2016-08-30 02:51:48  North Tonawanda, NY     Taylor     1.0   
16   2016-07-25 05:05:55       Germantown, MD    marilyn     1.0   
18   2016-07-17 22:27:13     Mays Landing, NJ        jim     1.0   
19   2016-07-05 15:35:17         Chicopee, MA      David     1.0   
23   2016-06-04 23:40:39          Brenham, TX     Alexis     1.0   
24   2016-06-01 03:52:58    Laguna Niguel, CA     farrah     1.0   
25   2016-05-14 22:23:58           Peoria, AZ      Alban     1.0   
29   2016-04-12 15:01:26            Plano, TX       Adam     1.0   
31   2017-01-03 16:46:44  Stevenson Ranch, CA        Joe     1.0   
33   2016-11-08 23:55:28        Knoxville, TN      K. L.     1.0   
34   2016-10-30 21:13:14         San Jose, CA      M. A.     1.0   
36   2016-10-25 21:57:07        Milwaukee, WI       Tony     1.0   
40   2016-08-30 02:51:48  North Tonawanda, NY     Taylor     1.0   
46   2016-07-25 05:05:55       Germantown, MD    marilyn     1.0   
48   2016-07-17 22:27:13     Mays Landing, NJ        jim     1.0   
49   2016-07-05 15:35:17         Chicopee, MA      David     1.0   
53   2016-06-04 23:40:39          Brenham, TX     Alexis     1.0   
54   2016-06-01 03:52:58    Laguna Niguel, CA     farrah     1.0   
55   2016-05-14 22:23:58           Peoria, AZ      Alban     1.0   
59   2016-04-12 15:01:26            Plano, TX       Adam     1.0   
65   2015-08-06 17:35:21        Jamesburg, NJ        Jim     1.0   
69   2015-07-15 17:11:04        Las Vegas, NV       rich     1.0   
70   2015-07-15 16:28:10       Palestine , WV     Shelby     1.0   
72   2015-07-03 13:09:49        Las Vegas, NV   Kimberly     1.0   
76   2015-06-05 07:06:52           Encino, CA    Vasiliy     1.0   
79   2015-05-11 16:16:00         Woodside, NY    Lindley     1.0   
...                  ...                  ...        ...     ...   
1242 2008-02-06 09:06:28           Sutton, MA        Sue     NaN   
1243 2008-02-03 14:21:08          Jamaica, NY    Richard     NaN   
1245 2008-01-25 16:49:16      Culver City, CA    Patrick     NaN   
1250 2007-12-19 13:15:25   Merritt Island, FL     Monica     NaN   
1252 2007-12-18 20:15:15       Scottsdale, AZ     Meghan     NaN   
1253 2007-12-13 13:18:19         Staunton, VA      Mindi     NaN   
1255 2007-12-10 14:28:40          Midvale, UT      Jamie     NaN   
1256 2007-12-09 03:23:56        Oceanside, CA          K     NaN   
1259 2007-12-07 13:52:24   Jeffersonville, IN     Daniel     NaN   
1261 2007-12-01 15:06:57          Roswell, NM      Nancy     NaN   
1262 2007-11-28 13:13:03    Fort Huachuca, AZ    Michael     NaN   
1263 2007-11-28 08:45:07  East Rutherford, NJ      Laura     NaN   
1264 2007-11-18 22:09:17       Sacramento, CA       Kate     NaN   
1265 2007-11-14 00:54:55         Petaluma, CA          J     NaN   
1267 2007-11-06 21:43:31      Kansas City, MO  Christine     NaN   
1273 2007-09-05 08:48:01      Goose Creek, SC     Leslie     NaN   
1274 2007-07-24 11:17:59      Tallahassee, FL      Steve     NaN   
1277 2007-05-04 12:29:10        Mt Laurel, NJ        Jim     NaN   
1278 2007-04-26 17:09:44      Casa Grande, AZ       Erin     NaN   
1280 2007-04-17 20:58:04        Grapevine, TX     Samuel     NaN   
1282 2007-04-11 19:35:04        Fullerton, CA     Julian     NaN   
1286 2007-04-04 12:09:30           Nutley, NJ   Margaret     NaN   
1287 2007-03-27 16:53:42          Hialeah, FL      Jason     NaN   
1294 2006-08-04 13:04:55            Parma, OH     Kelley     NaN   
1297 2006-06-12 17:26:07         Stockton, CA     Damali     NaN   
1300 2006-01-02 04:42:48      Los Angeles, CA       Paul     NaN   
1301 2005-10-19 12:24:55        Iowa City, IA     Joseph     NaN   
1305